In [5]:
import pandas as pd
import numpy as np

In [29]:
X = pd.read_csv('sp500/data.csv').reset_index()
X['date'] = pd.to_datetime(X['date'])
X = X.loc[(X['date'].dt.year == 2022) & (X['date'].dt.month == 12)]

Y = pd.read_csv('sp500/target.csv').reset_index()
Y['date'] = pd.to_datetime(Y['date'])
Y = Y.loc[(Y['date'].dt.year == 2023) & (Y['date'].dt.month == 12)].loc[~(Y['ticker'] == '^GSPC')][['ticker', 'class_2']].dropna()
common_tickers = set(Y['ticker']).intersection(X['ticker'])
X = X.drop(columns=['date'])

X = X.loc[X['ticker'].isin(common_tickers)].set_index('ticker').sort_index()
Y = Y.loc[Y['ticker'].isin(common_tickers)].set_index('ticker').sort_index()

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

scaler = StandardScaler()
clf = RandomForestClassifier(random_state=42)

steps = [
    ('scaler', scaler),
    ('clf', clf)
]
estimator = Pipeline(steps=steps)

param_grid = {
    'clf__n_estimators': [100, 200, 300],
    'clf__max_depth': [None, 10, 20, 30],
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [1, 2, 4],
    'clf__max_features': [None, 'sqrt', 'log2']
}

grid_search = GridSearchCV(
    estimator=estimator, 
    param_grid=param_grid,
    scoring='roc_auc',
    n_jobs=-2,
    cv=5,
    #verbose=2
)
res = grid_search.fit(X_train, Y_train)


/Users/micmur/PAD/.venv/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/micmur/PAD/.venv/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/micmur/PAD/.venv/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/micmur/PAD/.venv/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please ch

In [31]:
from sklearn.metrics import classification_report, confusion_matrix
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
    
print("\nBest parameters:", grid_search.best_params_)
print("\nBest cross-validation score:", grid_search.best_score_)
print("\nTest set score:", best_model.score(X_test, Y_test))    
print("\nClassification Report:")
print(classification_report(Y_test, y_pred))    
print("\nConfusion Matrix:")
print(confusion_matrix(Y_test, y_pred))

feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': best_model.named_steps['clf'].feature_importances_
})
feature_importance = feature_importance.sort_values('importance', ascending=False)
    
print("\nFeature Importances:")
print(feature_importance)
    



Best parameters: {'clf__max_depth': 10, 'clf__max_features': 'sqrt', 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 2, 'clf__n_estimators': 100}

Best cross-validation score: 0.6149238095238095

Test set score: 0.6

Classification Report:
              precision    recall  f1-score   support

           0       0.61      0.76      0.67        41
           1       0.58      0.41      0.48        34

    accuracy                           0.60        75
   macro avg       0.60      0.58      0.58        75
weighted avg       0.60      0.60      0.59        75


Confusion Matrix:
[[31 10]
 [20 14]]

Feature Importances:
                                      feature  importance
3                              asset_turnover    0.056297
33  total_depreciation_amortization_cash_flow    0.036244
40                              net_cash_flow    0.030495
12                       total_current_assets    0.030336
11                               cash_on_hand    0.029874
16                